### Quickstart: Compare runs, choose a model, and deploy it to a REST API

#### In this quickstart, you will:
- Run a hyper-parameter sweep on a training script
- Compare the results of the runs in the MLFlow UI
- Choose the best run an register it as model
- Deploy the model to a REST API
- Build a container image suitable for deployment to a cloud platform

In [49]:
import keras
import numpy as np
import pandas as pd
from hyperopt import STATUS_OK, Trials,fmin,hp,tpe
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import mlflow
from mlflow.models import infer_signature

#### Load the Dataset

In [50]:
data=pd.read_csv(
"https://raw.githubusercontent.com/mlflow/mlflow/master/tests/datasets/winequality-white.csv",
sep=";",
)
data

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.00100,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.99400,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.99510,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
...,...,...,...,...,...,...,...,...,...,...,...,...
4893,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,6
4894,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,5
4895,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6
4896,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,7


### SPlit the data into training,validation,test

In [51]:
train,test=train_test_split(data,test_size=0.25,random_state=42)

In [52]:
train

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
2835,6.3,0.25,0.22,3.30,0.048,41.0,161.0,0.99256,3.16,0.50,10.5,6
1157,7.8,0.30,0.29,16.85,0.054,23.0,135.0,0.99980,3.16,0.38,9.0,6
744,7.4,0.38,0.27,7.50,0.041,24.0,160.0,0.99535,3.17,0.43,10.0,5
1448,7.4,0.16,0.49,1.20,0.055,18.0,150.0,0.99170,3.23,0.47,11.2,6
3338,7.2,0.27,0.28,15.20,0.046,6.0,41.0,0.99665,3.17,0.39,10.9,6
...,...,...,...,...,...,...,...,...,...,...,...,...
4426,6.2,0.21,0.52,6.50,0.047,28.0,123.0,0.99418,3.22,0.49,9.9,6
466,7.0,0.14,0.32,9.00,0.039,54.0,141.0,0.99560,3.22,0.43,9.4,6
3092,7.6,0.27,0.52,3.20,0.043,28.0,152.0,0.99129,3.02,0.53,11.4,6
3772,6.3,0.24,0.29,13.70,0.035,53.0,134.0,0.99567,3.17,0.38,10.6,6


In [53]:
test

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
4656,6.00,0.29,0.41,10.80,0.048,55.0,149.0,0.99370,3.09,0.59,10.966667,7
3659,5.40,0.53,0.16,2.70,0.036,34.0,128.0,0.98856,3.20,0.53,13.200000,8
907,7.10,0.25,0.39,2.10,0.036,30.0,124.0,0.99080,3.28,0.43,12.200000,8
4352,7.30,0.28,0.35,1.60,0.054,31.0,148.0,0.99178,3.18,0.47,10.700000,5
3271,6.50,0.32,0.34,5.70,0.044,27.0,91.0,0.99184,3.28,0.60,12.000000,7
...,...,...,...,...,...,...,...,...,...,...,...,...
2614,6.15,0.21,0.37,3.20,0.021,20.0,80.0,0.99076,3.39,0.47,12.000000,5
755,7.10,0.28,0.44,1.80,0.032,32.0,107.0,0.99070,3.25,0.48,12.200000,7
518,5.90,0.13,0.28,1.90,0.050,20.0,78.0,0.99180,3.43,0.64,10.800000,6
3671,6.80,0.30,0.29,6.20,0.025,29.0,95.0,0.99071,3.03,0.32,12.900000,7


In [54]:
##train dataset
X_train=train.drop("quality",axis=1).values
y_train=train["quality"].values.ravel()

## test dataset
X_test=test.drop("quality",axis=1).values
y_test=test["quality"].values.ravel()

#validation dataset
train_x,valid_x,train_y,valid_y=train_test_split(X_train,y_train,test_size=0.2,random_state=42)
signature=infer_signature(train_x,train_y) #to capture input and output schema



## ANN MODEL

- Now to perform the noramalization we can take mean of entire data

In [55]:
def train_model(params,epochs,train_x,train_y,valid_x,valid_y,test_x,test_y):

    ## Define model architecture
    normalizer = keras.layers.Normalization()
    normalizer.adapt(train_x)

    model=keras.Sequential(
        [
            keras.Input([train_x.shape[1]]),
            normalizer,
            keras.layers.Dense(64,activation="relu"),
            keras.layers.Dense(1)
        ]
    )
    
    # Compile the model
    model.compile(
        optimizer=keras.optimizers.SGD(
        learning_rate=params["lr"],
        momentum=params["momentum"]
        ),
        loss="mean_squared_error",
        metrics=[keras.metrics.RootMeanSquaredError()]
    )

    #Train the ANN model with lr and momentum params with MLFLOW tracking
    with mlflow.start_run(nested=True):
        model.fit(
            train_x,
            train_y,
            validation_data=(valid_x,valid_y),
            epochs=epochs,
            batch_size=32,
            verbose=0
        )

        #Evaluate the model
        eval_result=model.evaluate(test_x,test_y,verbose=0)
        eval_rmse=eval_result[1]

        #Log the params
        mlflow.log_params(params)
        mlflow.log_metric("rmse",eval_rmse)

        # Log the model
        mlflow.tensorflow.log_model(model,"model",signature=signature)

        return {"loss": eval_rmse, "status": STATUS_OK, "model":model}

    


    

In [56]:
def objective(params):
    result=train_model(
        params,
        epochs=10,
        train_x=train_x,
        train_y=train_y,
        valid_x=valid_x,
        valid_y=valid_y,
        test_x=X_test,
        test_y=y_test,
    )
    return result

In [57]:
space={
    "lr":hp.loguniform("lr",np.log(1e-5),np.log(1e-1)),
    "momentum":hp.uniform("momentum",0.0,1.0)
}

In [58]:
mlflow.set_experiment("/wine-quality")
with mlflow.start_run():
    #conduct hyperparameter tuning with hyperopt
    trials=Trials() #This is gonna perform hyperparameter tuning
    best=fmin(
        fn=objective,
        space=space,
        algo=tpe.suggest,
        max_evals=4,
        trials=trials
    )
    #fetch the details of the best run
    best_run=sorted(trials.results,key=lambda x: x["loss"])[0]
    print(f"Best Parameters: {best}")
    print(f"Best RMSE: {best_run['loss']} ")



2025/10/29 13:23:14 INFO mlflow.tracking.fluent: Experiment with name '/wine-quality' does not exist. Creating a new experiment.


  0%|          | 0/4 [00:00<?, ?trial/s, best loss=?]

2025/10/29 13:23:19 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.



 25%|██▌       | 1/4 [00:13<00:39, 13.15s/trial, best loss: 0.7525510191917419]

2025/10/29 13:23:32 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.



 50%|█████     | 2/4 [00:26<00:26, 13.40s/trial, best loss: 0.7525510191917419]

2025/10/29 13:23:45 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.



 75%|███████▌  | 3/4 [00:40<00:13, 13.46s/trial, best loss: 0.7525510191917419]

2025/10/29 13:23:59 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.



100%|██████████| 4/4 [00:54<00:00, 13.64s/trial, best loss: 0.7525510191917419]
Best Parameters: {'lr': np.float64(0.00036521052744521883), 'momentum': np.float64(0.9566350744129403)}
Best RMSE: 0.7525510191917419 


### Infrencing the model

In [61]:
from mlflow.models import validate_serving_input

model_uri = "runs:/21c7b8b13c2844d28c51af2cc1bd6ee9/model"

from mlflow.models import convert_input_example_to_serving_input

serving_payload=convert_input_example_to_serving_input(X_test)

validate_serving_input(model_uri,serving_payload)

/home/sushant/Desktop/Github_Study/MLFLOW_PROJECTS/venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step


array([[5.959608 ],
       [7.031024 ],
       [6.436517 ],
       ...,
       [6.3197455],
       [6.8456182],
       [5.730827 ]], shape=(1225, 1), dtype=float32)

In [62]:
#load the model for inference as a PyFunc model
loaded_model=mlflow.pyfunc.load_model(model_uri)

#predict on a pandas dataframe
import pandas as pd
loaded_model.predict(pd.DataFrame(X_test))

39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step


array([[5.959608 ],
       [7.031024 ],
       [6.436517 ],
       ...,
       [6.3197455],
       [6.8456182],
       [5.730827 ]], shape=(1225, 1), dtype=float32)

## Register in the model registry

In [63]:
mlflow.register_model(model_uri,"wine-quality-model")

Successfully registered model 'wine-quality-model'.
2025/10/29 14:54:32 WARNING mlflow.tracking._model_registry.fluent: Run with id 21c7b8b13c2844d28c51af2cc1bd6ee9 has no artifacts at artifact path 'model', registering model based on models:/m-cd14121af1994701ad5d6ac70a864d57 instead
Created version '1' of model 'wine-quality-model'.


<ModelVersion: aliases=[], creation_timestamp=1761729872821, current_stage='None', deployment_job_state=None, description=None, last_updated_timestamp=1761729872821, metrics=[<Metric: dataset_digest=None, dataset_name=None, key='rmse', model_id='m-cd14121af1994701ad5d6ac70a864d57', run_id='21c7b8b13c2844d28c51af2cc1bd6ee9', step=0, timestamp=1761724399097, value=0.7525510191917419>], model_id='m-cd14121af1994701ad5d6ac70a864d57', name='wine-quality-model', params={'lr': '0.00036521052744521883', 'momentum': '0.9566350744129403'}, run_id='21c7b8b13c2844d28c51af2cc1bd6ee9', run_link=None, source='models:/m-cd14121af1994701ad5d6ac70a864d57', status='READY', status_message=None, tags={}, user_id=None, version=1>